### ORB OpenCV using Voting Classifier

**AIM:** Record keypoints and description parameters from at-least 5 to 10 frames during "SAVE MODEL" phase. On loading the model back and checking for matching the parameters, check for each of the 5-10 frames and have a "VOTING" model to decide whether the features matched or not.

In [1]:
import numpy as np
import pickle
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def pickle_keypoints(keypoints, descriptors):
    i = 0
    temp_array = []
    for point in keypoints:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, point.class_id, descriptors[i])     
        ++i
        temp_array.append(temp)
    return temp_array

In [3]:
def unpickle_keypoints(array):
    keypoints = []
    descriptors = []
    for point in array:
        temp_feature = cv2.KeyPoint(x=point[0][0],y=point[0][1],_size=point[1], _angle=point[2], _response=point[3], _octave=point[4], _class_id=point[5])
        temp_descriptor = point[6]
        keypoints.append(temp_feature)
        descriptors.append(temp_descriptor)
    return keypoints, np.array(descriptors)

In [4]:
cap = cv2.VideoCapture(0)

saved_data = []
#saved_des = []

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Initiate STAR detector
    orb = cv2.ORB_create(400)
    
    # find the keypoints with ORB
    kp = orb.detect(gray,None)
    
    # compute the descriptors with ORB
    kp, des = orb.compute(gray, kp)
    
    # draw only keypoints location,not size and orientation
    img = cv2.drawKeypoints(gray,kp,None, flags=0)
    
    cv2.imshow('dst',img)
    
    if cv2.waitKey(33) & 0xFF == ord('s'):
        print("Capturing ORB Features...")
        saved_features = pickle_keypoints(kp, des)
        saved_data.append(saved_features)
        #pickle.dump(saved_features, open("saved_features.p", "wb"))
        #print("ORB Features Saved...")
        #saved_features.append(kp)
        #saved_des.append(des)
        #break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Saving ORB Features...\n")
        pickle.dump(saved_data, open("saved_features.p", "wb"))
        print("ORB Features Saved...")
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Capturing ORB Features...
Capturing ORB Features...
Capturing ORB Features...
Capturing ORB Features...
Capturing ORB Features...
Saving ORB Features...

ORB Features Saved...


In [5]:
len(saved_features)

400

In [6]:
saved_features

[((235.0, 31.0),
  31.0,
  232.4183807373047,
  0.000541234971024096,
  0,
  -1,
  array([235, 151,  99, 143, 228, 148, 250, 117, 121, 232,  89, 231, 249,
         252, 190, 167, 254, 111, 183, 239, 213, 250, 190, 236,  62, 120,
         238, 234,  15, 166, 242, 253], dtype=uint8)),
 ((255.0, 31.0),
  31.0,
  289.54010009765625,
  0.000361662037903443,
  0,
  -1,
  array([235, 151,  99, 143, 228, 148, 250, 117, 121, 232,  89, 231, 249,
         252, 190, 167, 254, 111, 183, 239, 213, 250, 190, 236,  62, 120,
         238, 234,  15, 166, 242, 253], dtype=uint8)),
 ((298.0, 96.0),
  31.0,
  228.86724853515625,
  0.000338190991897136,
  0,
  -1,
  array([235, 151,  99, 143, 228, 148, 250, 117, 121, 232,  89, 231, 249,
         252, 190, 167, 254, 111, 183, 239, 213, 250, 190, 236,  62, 120,
         238, 234,  15, 166, 242, 253], dtype=uint8)),
 ((296.0, 96.0),
  31.0,
  230.62335205078125,
  0.0003559958713594824,
  0,
  -1,
  array([235, 151,  99, 143, 228, 148, 250, 117, 121, 232,  89,

### TEST 1

In [7]:
# cap = cv2.VideoCapture(0)

# good_match_points = []
# saved_des2 = []

# #Retrieve Saved Keypoint Features
# keypoints_database = pickle.load(open( "saved_features.p", "rb" ))
# kp1, des1 = unpickle_keypoints(keypoints_database)

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
#     # Initiate STAR detector
#     orb = cv2.ORB_create(400)
    
#     # find the keypoints with ORB
#     kp2 = orb.detect(gray,None)
    
#     # compute the descriptors with ORBq
#     kp2, des2 = orb.compute(gray, kp2)
    
#     # draw only keypoints location,not size and orientation
#     img1 = cv2.drawKeypoints(gray,kp2,None, flags=0)
    
#     cv2.imshow('dst',img1)
    
#     # create BFMatcher object
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
#     # Match Saved Data Points with New Points
#     if cv2.waitKey(33) & 0xFF == ord('c'):
#         print("Checking for Match...\n")
#         # Match descriptors.
#         matches = bf.match(des1,des2)
#         # Sort them in the order of their distance.
#         matches = sorted(matches, key = lambda x:x.distance)
#         break
        
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

### TEST 2

In [8]:
def tune_ratio(matches, ratio=0.75):
    print("Tuning Ratio....")
    good = []
    for m,n in matches:
        if m.distance < ratio * n.distance:
            good.append([m])
    if (len(good) > 0):
        print("SCENE MATCHED....")
        return good
    else:
        if (ratio > 0.30):
            ratio = ratio - 0.05
            tune_ratio(matches, ratio)
        else:
            print("Unable to Tune, Exiting....")
            return good
    return good

In [9]:
cap = cv2.VideoCapture(0)

match_result = []

#Retrieve Saved Keypoint Features
keypoints_database = pickle.load(open( "saved_features.p", "rb" ))
#kp1, des1 = unpickle_keypoints(keypoints_database)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Initiate STAR detector
    orb = cv2.ORB_create(400)
    
    # find the keypoints with ORB
    kp2 = orb.detect(gray,None)
    
    # compute the descriptors with ORBq
    kp2, des2 = orb.compute(gray, kp2)
    
    # draw only keypoints location,not size and orientation
    img1 = cv2.drawKeypoints(gray,kp2,None, flags=0)
    
    cv2.imshow('dst',img1)
    
    # Matcher
    FLANN_INDEX_LSH = 6
    index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6, # 12
                   key_size = 12,     # 20
                   multi_probe_level = 1) #2
    search_params = dict(checks=60) 
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    # Match Saved Data Points with New Points
    if cv2.waitKey(33) & 0xFF == ord('c'):
        print("Checking for Match...\n")
        
        for i in range(5):
            print("Comparing Current Scene with Recorded Scene {}".format(i+1))
            kp1, des1 = unpickle_keypoints(keypoints_database[i])
            # Match descriptors.
            matches = flann.knnMatch(des1,des2,k=2)

            # Apply ratio test
            good = []
            for m,n in matches:
                if m.distance < 0.75 * n.distance:
                    good.append([m])

            print("Number of Good Matches: ", len(good))
            if (len(good) > 0):
                print("SCENE MATCHED....")
                # If Frames Match, it's a "1"
                match_result.append(1)
            else:
                print("SCENE NOT MATCHED....")
                # If Frames don't Match, it's a "0"
                match_result.append(0)
                #good = tune_ratio(matches, ratio=0.75)
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Checking for Match...

Comparing Current Scene with Recorded Scene 1
Number of Good Matches:  400
SCENE MATCHED....
Comparing Current Scene with Recorded Scene 2
Number of Good Matches:  400
SCENE MATCHED....
Comparing Current Scene with Recorded Scene 3
Number of Good Matches:  0
SCENE NOT MATCHED....
Comparing Current Scene with Recorded Scene 4
Number of Good Matches:  400
SCENE MATCHED....
Comparing Current Scene with Recorded Scene 5
Number of Good Matches:  400
SCENE MATCHED....


In [10]:
match_result

[1, 1, 0, 1, 1]

In [11]:
unique, counts = np.unique(match_result, return_counts=True)

In [12]:
dict(zip(unique, counts))

{0: 1, 1: 4}

In [13]:
total_outputs = len(match_result)
num_ones = np.count_nonzero(match_result, axis=0)
num_zeros = total_outputs - num_ones

if (num_ones > num_zeros):
    print("Scene Matched Successfully....")
else:
    print("Scene Not Matched....")

Scene Matched Successfully....
